In [ ]:
!pip install python_speech_features

In [25]:
!pip install sounddevice

In [ ]:
!pip install pyaudio

In [ ]:
pip install librosa scikit-learn python_speech_features sounddevice tqdm

In [28]:
pip install sounddevice

In [38]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from python_speech_features import mfcc
from tqdm import tqdm
#import sounddevice as sd

In [39]:
def extract_mfcc_features(audio, sr=16000, num_cepstral=13, frame_length=0.02, frame_stride=0.02,
                           num_filters=32, fft_length=256, normalization_window=101, preemphasis_coeff=0.98):
    # Apply pre-emphasis
    audio = np.append(audio[0], audio[1:] - preemphasis_coeff * audio[:-1])

    # Compute the spectrogram
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=num_filters,
                                                 hop_length=int(frame_stride * sr),
                                                 win_length=int(frame_length * sr),
                                                 n_fft=fft_length, power=2.0)

    # Apply logarithm to the spectrogram to get log-mel energies
    log_mel_energy = np.log(spectrogram + 1e-9)

    # Extract MFCC features
    mfcc_features = librosa.feature.mfcc(S=log_mel_energy, n_mfcc=num_cepstral)

    # Apply mean and variance normalization across time
    mfcc_features = librosa.util.normalize(mfcc_features, norm=2, axis=0, fill=True)

    return mfcc_features

In [42]:
# Function to load data and preprocess
def preprocess_data(data_dir, test_size=0.2):
    labels = []
    features = []

    for class_label in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_label)
        for file_name in tqdm(os.listdir(class_path), desc=f"Processing {class_label}"):
            file_path = os.path.join(class_path, file_name)
            audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=2.0, offset=0.0)
            mfcc_features = extract_mfcc_features(audio)
            features.append(mfcc_features)
            labels.append(class_label)

    features = np.array(features)
    labels = np.array(labels)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=42)

    return X_train, X_test, y_train, y_test,labels

In [45]:
# Function to train the model

# Determine the number of classes

def train_model(features, labels, num_classes):
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
        layers.Reshape((13, -1)),
        layers.Conv1D(8, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.25),
        layers.Conv1D(16, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

    return model, encoder

# Define your data directory
data_dir = "/content/drive/MyDrive/hackathon"

# Preprocess data
features, labels = preprocess_data(data_dir)
num_classes = len(set(labels))

# Train the model
trained_model, label_encoder = train_model(features, labels)

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/hackathon/.DS_Store'

# **Inferencing**


In [34]:
# Real-time inference
def real_time_inference(model, encoder):
    duration = 1  # seconds
    sample_rate = 16000

    print("Listening for real-time inference...")

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=int(sample_rate * duration))

    while True:
        # Record audio from the microphone
        audio_input = np.frombuffer(stream.read(int(sample_rate * duration), exception_on_overflow=False), dtype=np.int16)

        # Extract MFCC features from the audio input
        mfcc_features = extract_features(audio_input)

        # Perform real-time prediction
        prediction = model.predict([mfcc_features])

        # Inverse transform to get the predicted class label
        class_label = encoder.inverse_transform(prediction)[0]

        # Print the predicted class label
        print(f"Predicted class: {class_label}")

    stream.stop_stream()
    stream.close()
    p.terminate()

# Define your data directory
data_directory = "/Users/snigdhaasthana/Desktop/hackathon"

# Preprocess data
features, labels = preprocess_data(data_directory)

# Train the model
trained_model, label_encoder = train_model(features, labels)

# Real-time inference
real_time_inference(trained_model, label_encoder)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/snigdhaasthana/Desktop/hackathon'